In [8]:
import pandas as pd

from automated_llm_eval.chat_model import ChatModel
from automated_llm_eval.utils import ProgressBar

# Instantiate wrapper around OpenAI's API
model = ChatModel(model="gpt-3.5-turbo-1106")
# model = ChatModel(model="gpt-4-1106-preview")

In [2]:
# Make API call, get response message.  Note: `output_format = "simple"`
response_message = model.create_chat_completion(
    system_message="You are a joke telling machine.",
    user_message="Tell me something about apples.",
    output_format="simple",
)
print(response_message)

Why did the apple stop in the middle of the road?

Because it ran out of juice!


In [3]:
# Make API call, get original ChatCompletion object.  Note: `output_format = None`
response = model.create_chat_completion(
    system_message="You are a joke telling machine.",
    user_message="Tell me something about apples.",
    output_format=None,
)
print(response)

ChatCompletion(id='chatcmpl-8JAf8828XLtpMWxtK4usppfzKbHHx', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Why did the apple stop in the middle of the road?\n\nBecause it ran out of juice!', role='assistant', function_call=None, tool_calls=None))], created=1699579910, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_eeff13170a', usage=CompletionUsage(completion_tokens=19, prompt_tokens=24, total_tokens=43))


In [4]:
# Make API call, get response packaged with input + metadata.  Note: `output_format = "message_bundle"`
message_bundle = model.create_chat_completion(
    system_message="You are a joke telling machine.",
    user_message="Tell me something about apples.",
    output_format="message_bundle",
)
print(message_bundle)

MessageBundle(id='chatcmpl-8JAf8MQG221eWTCSrdxVHLodZcjPp', system_message='You are a joke telling machine.', user_message='Tell me something about apples.', response_message='Why did the apple stop in the middle of the road?\n\nBecause it ran out of juice!', created_time=1699579910, model='gpt-3.5-turbo-1106', total_tokens=43, prompt_tokens=24, completion_tokens=19, seed=42, temperature=0.9, top_p=0.9, max_tokens=None)


In [5]:
# Make API call, get MessageBundle as a dict.  Note: `output_format = "message_bundle_dict"`
message_bundle_dict = model.create_chat_completion(
    system_message="You are a joke telling machine.",
    user_message="Tell me something about apples.",
    output_format="message_bundle_dict",
)
print(message_bundle_dict)

{'id': 'chatcmpl-8JAfA3TRNUU4W6wGbvDTIFJSuWqoz', 'system_message': 'You are a joke telling machine.', 'user_message': 'Tell me something about apples.', 'response_message': 'Why did the apple stop in the middle of the road?\n\nBecause it ran out of juice!', 'created_time': 1699579912, 'model': 'gpt-3.5-turbo-1106', 'total_tokens': 43, 'prompt_tokens': 24, 'completion_tokens': 19, 'seed': 42, 'temperature': 0.9, 'top_p': 0.9, 'max_tokens': None}


In [6]:
# Message bundle dict can be converted into pandas Series easily
s = pd.Series(message_bundle_dict)
s

id                              chatcmpl-8JAfA3TRNUU4W6wGbvDTIFJSuWqoz
system_message                         You are a joke telling machine.
user_message                           Tell me something about apples.
response_message     Why did the apple stop in the middle of the ro...
created_time                                                1699579912
model                                               gpt-3.5-turbo-1106
total_tokens                                                        43
prompt_tokens                                                       24
completion_tokens                                                   19
seed                                                                42
temperature                                                        0.9
top_p                                                              0.9
max_tokens                                                        None
dtype: object

In [7]:
# Multiple message bundle dicts can be converted into pandas DataFrame
responses = []
with ProgressBar() as p:
    for _ in p.track(range(5)):
        response = model.create_chat_completion(
            system_message="You are a joke telling machine.",
            user_message="Tell me something about apples.",
            output_format="message_bundle_dict",
            temperature=0.4,
            seed=None,
        )
        responses += [response]

df = pd.DataFrame(responses)
df

Output()

,id,system_message,user_message,response_message,created_time,model,total_tokens,prompt_tokens,completion_tokens,seed,temperature,top_p,max_tokens
0,chatcmpl-8JAfARi100fvA5tYzrd9gy7ksq5i0,You are a joke telling machine.,Tell me something about apples.,Sure! Did you hear about the apple that joined...,1699579912,gpt-3.5-turbo-1106,44,24,20,None,0.4,0.9,None
1,chatcmpl-8JAfCwNvCs7FcI6Qhs4DYyiK4nVW5,You are a joke telling machine.,Tell me something about apples.,Why did the apple go to the doctor? Because it...,1699579914,gpt-3.5-turbo-1106,41,24,17,None,0.4,0.9,None
2,chatcmpl-8JAfEZsZv7ybliUCdXzEF3UaC2ZL3,You are a joke telling machine.,Tell me something about apples.,Why did the apple go to the doctor? Because it...,1699579916,gpt-3.5-turbo-1106,41,24,17,None,0.4,0.9,None
3,chatcmpl-8JAfE7fpmxVztxnSauyKUmynThWVM,You are a joke telling machine.,Tell me something about apples.,Sure! Did you hear about the apple who went to...,1699579916,gpt-3.5-turbo-1106,47,24,23,None,0.4,0.9,None
4,chatcmpl-8JAfGhQErNcm3ZUbvoO7wBWmWA9Y7,You are a joke telling machine.,Tell me something about apples.,Why did the apple go to the doctor? Because it...,1699579918,gpt-3.5-turbo-1106,41,24,17,None,0.4,0.9,None
